# YOLO (You Only Look Once)

paper的翻译笔记

http://arxiv.org/abs/1506.02640

2. Unified Detection

We unify the separate components of object detection into a single neural network. Our network uses features from the entire image to predict each bounding box. It also predicts all bounding boxes across all classes for an image simultaneously. This means our network reasons globally about the full image and all the objects in the image. The YOLO design enables end-to-end training and realtime speeds while maintaining high average precision.

我们把物体检测中用到的独立的几个组件统一成了一个单独的神经网络。我们的网络使用来自整个图片的特征去预测每个边界框。同时它还预测每个边界框和每种物体分类。这意味着我们的网络对整个图片和图片中所有的物体是从全局上理解的。YOLO的设计在保持了高平均准确率的同时，也实现了端到端的训练和实时的速度。

Our system divides the input image into an S × S grid. If the center of an object falls into a grid cell, that grid cell is responsible for detecting that object.

我们的系统把输入图片划分成S x S的网格。如果一个物体的中心落在了某一个网格中，这个网格就负责来检测这个物体。

Each grid cell predicts B bounding boxes and confidence scores for those boxes. These confidence scores reflect how
confident the model is that the box contains an object and also how accurate it thinks the box is that it predicts. Formally we define confidence as Pr(Object) ∗ IOU. If no pred object exists in that cell, the confidence scores should be zero. Otherwise we want the confidence score to equal the intersection over union (IOU) between the predicted box and the ground truth.

每一个网格会预测出 B 个边界框和边界框对应的信心值。这个值反应了这个边界框中包含了一个物体，并且这个边界框有多准确的信心。形式上定义这个信心为 Pr(Object) * IOU。如果预测认为框中没有物体，那么这个值就是0。否则我们希望这个值等于预测框和真实框的IOU。

- Pr(Object) 框中有一个物体
- IOU：intersection over union，代表了边界框的准确度。真实的边界框和预测的边界框，这两个框的交集/这两个的并集。

Each bounding box consists of 5 predictions: x, y, w, h, and confidence. The (x, y) coordinates represent the center of the box relative to the bounds of the grid cell. The width and height are predicted relative to the whole image. Finally the confidence prediction represents the IOU between the predicted box and any ground truth box.

每个边界框包含5个预测值：x，y，w，h和信心。坐标（x，y）表示边界框的中心点（相对于当前的网格）。宽和高是相对于整个图片来说的。最后信心表示预测框和真实框的IOU。

Each grid cell also predicts C conditional class probabilities, Pr(Class_i |Object). These probabilities are conditioned on the grid cell containing an object. We only predict one set of class probabilities per grid cell, regardless of the number of boxes B.

每个网格还会预测C个条件概率，Pr(Class_i | Object)。是在网格包含一个物体的条件下某个分类的概率。我们对每个网格只预测了集合中每个分类的概率，而不管这个网格有B个边界框。

At test time we multiply the conditional class probabilities and the individual box confidence predictions,
Pr(Class_i|Object) ∗ Pr(Object) ∗ IOU = Pr(Classi) ∗ IOU which gives us class-specific confidence scores for each box. These scores encode both the probability of that class appearing in the box and how well the predicted box fits the object.

在测试的时候，我们把分类的条件概率和每个编辑框的信心相乘，对每个框我们就知道它包含的分类的信心。这个数值既代表了某个分类物体出现在框中的概率，也代表了框是否很好地“框住”了这个物体。

Figure 2: The Model. Our system models detection as a regression problem. It divides the image into an S × S grid and for each grid cell predicts B bounding boxes, confidence for those boxes, and C class probabilities. These predictions are encoded as an S × S × (B ∗ 5 + C) tensor.

For evaluating YOLO on PASCAL VOC, we use S = 7, B = 2. PASCAL VOC has 20 labelled classes so C = 20. Our final prediction is a 7 × 7 × 30 tensor.

在PASCAL VOC上来评估YOLO时，我们使用了 S = 7，B = 2. PASCAL VOC包含20个分类，所以 C = 20. 我们最终的预测值就是一个 7 x 7 x 30的张量。

2.1. Network Design

We implement this model as a convolutional neural network and evaluate it on the PASCAL VOC detection dataset [9]. The initial convolutional layers of the network extract features from the image while the fully connected layers predict the output probabilities and coordinates.

我们用一个卷积网络来实现这个模型并用PASCAL VOC检测数据库来评估。开始的卷积层用来从图片提取特征，后面的全连接层用来预测概率和坐标。

Our network architecture is inspired by the GoogLeNet model for image classification [34]. Our network has 24 convolutional layers followed by 2 fully connected layers. Instead of the inception modules used by GoogLeNet, we simply use 1 × 1 reduction layers followed by 3 × 3 convolutional layers, similar to Lin et al [22]. The full network is shown in Figure 3.

这种网络结构的设计受到了GoogLeNet图片分类模型的启发。我们的网络有24个卷积层，然后两个全连接层。我们使用1x1的reduction layer后跟3x3的卷积层，而不是GoogLeNet中的inception模块。图片3显示了网络的整体结构。

We also train a fast version of YOLO designed to push the boundaries of fast object detection. Fast YOLO uses a neural network with fewer convolutional layers (9 instead of 24) and fewer filters in those layers. Other than the size of the network, all training and testing parameters are the same between YOLO and Fast YOLO.

我们还设计了另一个快速版本的YOLO网络来做物体的边界检测。Fast YOLO使用一个小一点的神经网络，它有9个而不是24个卷积层，并且每个卷积层的卷积核少一些。除了网络的大小以外，训练和测试中的别的参数都和YOLO一样。

![Figure 3](yolo_arch.png)

Figure 3: The Architecture. Our detection network has 24 convolutional layers followed by 2 fully connected layers. Alternating 1 × 1 convolutional layers reduce the features space from preceding layers. We pretrain the convolutional layers on the ImageNet classification task at half the resolution (224 × 224 input image) and then double the resolution for detection.

检测网络有24个conv层和2个fc层。交替地使用1x1的卷积层来减低前一层的特征空间。我们使用一半大小的ImageNet分类任务来预训练卷积层（224 x 224的图片输入）然后增大分辨率来做检测。

The final output of our network is the 7 × 7 × 30 tensor of predictions.

网络的输出是一个7 x 7 x 30的预测值。

解释一下图片3里表达形式：

- `Conv. Layer 7x7x64-s-2` 表示64个7x7的kernel，stride=2的卷积层
- `Maxpool Layer 2x2-s-2` 表示2x2 stride=2的max pooling层

2.2. Training

We pretrain our convolutional layers on the ImageNet 1000-class competition dataset [30]. For pretraining we use the first 20 convolutional layers from Figure 3 followed by a average-pooling layer and a fully connected layer. We train this network for approximately a week and achieve a single crop top-5 accuracy of 88% on the ImageNet 2012 validation set, comparable to the GoogLeNet models in Caffe’s Model Zoo [24]. We use the Darknet framework for all training and inference [26].

我们在ImageNet-1000分类竞赛数据集对网络做预训练。在预训练中我们使用了前20个conv层，后跟一个平均池化层和一个fc层。大约训练了一周以后，在2012验证集上，a single crop top-5准确率达到了88%。我们使用Darknet框架来做训练和推断。

We then convert the model to perform detection. Ren et al. show that adding both convolutional and connected layers to pretrained networks can improve performance [29]. Following their example, we add four convolutional layers and two fully connected layers with randomly initialized weights. Detection often requires fine-grained visual information so we increase the input resolution of the network from 224 × 224 to 448 × 448.

之后我们使用这个模型来做检测。Ren et al.说明在预训练网络中添加conv和fc可以提升性能。按照他们给的例子，我们添加了4个conv层和2个fc层，并随机初始化这些参数。检测经常需要更细粒度的视觉信息，所以我们增加了输入的分辨率，把网络的输入从224x224变成了448x448。

Our final layer predicts both class probabilities and bounding box coordinates. We normalize the bounding box width and height by the image width and height so that they fall between 0 and 1. We parametrize the bounding box x and y coordinates to be offsets of a particular grid cell location so they are also bounded between 0 and 1.

我们的最后一层同时预测了类别信息和位置信息。我们把边框的高和宽对图片的大小做归一化到0到1之间。边框的x，y坐标，我们表示为到一个特定网格的偏移，所以也0到1之间的数。

We use a linear activation function for the final layer and all other layers use the following leaky rectified linear activation:

网络的最后一层是线性激励函数，而其他的层用的是下面的这个leaky-relu激励：

$$
\phi(x) =
\begin{cases}
    x, & if x > 0 \\
    0.1x, & otherwise
\end{cases}
$$

We optimize for sum-squared error in the output of our model. We use sum-squared error because it is easy to optimize, however it does not perfectly align with our goal of maximizing average precision. It weights localization error equally with classification error which may not be ideal. Also, in every image many grid cells do not contain any object. This pushes the “confidence” scores of those cells towards zero, often overpowering the gradient from cells that do contain objects. This can lead to model instability, causing training to diverge early on.

我们使用平方和误差来优化模型。选择平方和误差是因为训练简单，尽管它不能很好的和我们的目标相匹配。我们的目标是最大化平均准确率。这种优化相等的对待分类误差并不是很理想。而且，在图片中很多的网格里并不包含任何物体。这样就使得这些网格的信心输出为0，经常会导致那些包含物体的网格的梯度过大。这样导致了模型不稳定，从而使训练较早就发散了。

To remedy this, we increase the loss from bounding box coordinate predictions and decrease the loss from confidence predictions for boxes that don’t contain objects. We use two parameters, λcoord and λnoobj to accomplish this. We set λcoord = 5 and λnoobj = .5.

为了解决这个问题，我们增大了边界框预测的损失值，同时减少不包含物体的那些网格的损失值。我们使用两个参数$\lambda_{coord}$和$\lambda_{noobj}$ 来控制。设$\lambda_{coord} = 5$和$\lambda_{noobj} = .5$

Sum-squared error also equally weights errors in large boxes and small boxes. Our error metric should reflect that small deviations in large boxes matter less than in small boxes. To partially address this we predict the square root of the bounding box width and height instead of the width and height directly.

平方和误差在对待大的边界框和小边界框也是同等的。而我们的误差需要反映出，一个小的偏差对大框的影响比对小框的影响要小。为了部分解决这个问题，我们预测边界框高和宽的平方根，而不是高和宽本身。

YOLO predicts multiple bounding boxes per grid cell. At training time we only want one bounding box predictor to be responsible for each object. We assign one predictor to be “responsible” for predicting an object based on which prediction has the highest current IOU with the ground truth. This leads to specialization between the bounding box predictors. Each predictor gets better at predicting certain sizes, aspect ratios, or classes of object, improving overall recall.

每个网格会预测多个边界框。在训练阶段，我们希望每个物体只由一个网格来预测。我们指定具有最高的IOU的那个预测者来负责预测这个物体。这就导致了边界框和预测者之间建立了专门联系。每个预测者在预测大小、比率或者分类的时候更好，就改进了整体的召回。

During training we optimize the following, multi-part loss function:

以下是整体的损失函数：

$$
\lambda_{coord} \sum_{i=0}^{S^2} \sum_{j=0}^B \mathbb{1}_{ij}^{obj} [(x_i - \hat{x}_i)^2 + (y_i - \hat{y}_i)^2] \\
+ \lambda_{coord} \sum_{i=0}^{S^2} \sum_{j=0}^B \mathbb{1}_{ij}^{obj} [(\sqrt{w_i} - \sqrt{\hat{w}_i})^2 + (\sqrt{h_i} - \sqrt{\hat{h}_i})^2] \\
+ \sum_{i=0}^{S^2} \sum_{j=0}^B \mathbb{1}_{ij}^{obj} (C_i - \hat{C}_i)^2 \\
+ \lambda_{noobj} \sum_{i=0}^{S^2} \sum_{j=0}^B \mathbb{1}_{ij}^{noobj} (C_i - \hat{C}_i)^2 \\
+ \sum_{i=0}^{S^2} \mathbb{1}_i^{obj} \sum_{c \in classes} (p_i(c) - \hat{p}_i(c)) ^2
$$

where $\mathbb{1}_i^{obj}$ denotes if object appears in cell i and $\mathbb{1}_{ij}^{obj}$ denotes that the jth bounding box predictor in cell i is “responsible” for that prediction.

- $\mathbb{1}_i^{obj}$ 表示`obj`是否出现在第`i`个网格里
- $\mathbb{1}_{ij}^{obj}$ 表示第`i`个网格里的第`j`个边界框“负责”预测

这个函数由5个部分求和而来：
1. 当obj出现在网格i里时，预测x，y的误差
2. 当obj出现在网格i里时，预测w，h的误差
3. 当obj出现在网格i里时，预测分类的误差
4. 

Note that the loss function only penalizes classification error if an object is present in that grid cell (hence the conditional class probability discussed earlier). It also only penalizes bounding box coordinate error if that predictor is “responsible” for the ground truth box (i.e. has the highest IOU of any predictor in that grid cell).

损失函数中，只有当物体出现在一个网格中的时候，才会计算分类的误差；只有当某个边界框负责预测的时候，才计算边界框的误差。

We train the network for about 135 epochs on the training and validation data sets from PASCAL VOC 2007 and 2012. When testing on 2012 we also include the VOC 2007 test data for training. Throughout training we use a batch size of 64, a momentum of 0.9 and a decay of 0.0005.

我们使用PASCAL VOC 2007和2012训练集和验证集训练了大约135个周期。当在2012上进行测试的时候，我们还将2007的测试数据也用来进行了训练。整个训练的过程中使用64大小的批，动量0.9和0.0005的衰减。

Our learning rate schedule is as follows: For the first epochs we slowly raise the learning rate from 10−3 to 10−2. If we start at a high learning rate our model often diverges due to unstable gradients. We continue training with 10−2 for 75 epochs, then 10−3 for 30 epochs, and finally 10−4 for 30 epochs.

我们的学习率的方案如下：最开始的几个周期，我们慢慢将学习率从$10^{-3}$变为$10^{-2}$。如果我们使用一个过高的学习率开始，会导致不稳定和发散的梯度。接着用$10^{-2}$继续训练75个周期，然后用$10^{-3}$训练30个周期，最后$10^{-4}$训练30个周期。

To avoid overfitting we use dropout and extensive data augmentation. A dropout layer with rate = .5 after the first connected layer prevents co-adaptation between layers [18]. For data augmentation we introduce random scaling and translations of up to 20% of the original image size. We also randomly adjust the exposure and saturation of the image by up to a factor of 1.5 in the HSV color space.

为了避免过拟合，我们使用dropout和大量的数据增强。使用.5保留率的dropout在the first connected layer来防止层之间的co-adaptation。数据增强方面，我们使用最多20%的随机放大、缩小和变化操作。并在HSV空间上对亮度和饱和度进行随机调整，使用的最大因子为1.5.



2.3. Inference

Just like in training, predicting detections for a test image only requires one network evaluation. On PASCAL VOC the network predicts 98 bounding boxes per image and class probabilities for each box. YOLO is extremely fast at test time since it only requires a single network evaluation, unlike classifier-based methods.

就像训练的时候一样，在一个测试的图片上进行预测的时候也只需要一个网络。在PASCAL VOC上网络会对每个图片产生98个框和对应的分类概率。YOLO不像其他基于分类器的方法，在测试的时候非常的快，是因为它只需要一次网络求值。

The grid design enforces spatial diversity in the bounding box predictions. Often it is clear which grid cell an object falls in to and the network only predicts one box for each object. However, some large objects or objects near the border of multiple cells can be well localized by multiple cells. Non-maximal suppression can be used to fix these multiple detections. While not critical to performance as it is for R-CNN or DPM, non-maximal suppression adds 2-3% in mAP.

网格的设计强化了空间的多样性。大部分时候一个物体落入哪个网格是清楚的，网络只会对每个物体只会预测出一个边框。但是一些较大的物体或者在多个网格的边界上的物体会被好几个网格识别。不做最大值限制可以解决这种多个检测。non-maximal suppression对mAP有2%-3%的提升。

2.4. Limitations of YOLO

YOLO imposes strong spatial constraints on bounding box predictions since each grid cell only predicts two boxes and can only have one class. This spatial constraint limits the number of nearby objects that our model can predict. Our model struggles with small objects that appear in groups, such as flocks of birds.

YOLO在空间上做了很强的约束，一个网格只能产生两个边框和只有一个分类。这种约束限制了模型能够预测的临近物体的个数。我们的模型在一组小物体的情况下就很为难，例如一群鸟。

Since our model learns to predict bounding boxes from data, it struggles to generalize to objects in new or unusual aspect ratios or configurations. Our model also uses relatively coarse features for predicting bounding boxes since our architecture has multiple downsampling layers from the input image.

因为我们的模型是从数据中来学习预测边框，不太好泛化到新的物体或者不太常见的比例或者结构。模型也使用了相对粗粒度的特征来预测，因为对输入图片进行了很多的下采样。

Finally, while we train on a loss function that approximates detection performance, our loss function treats errors the same in small bounding boxes versus large bounding boxes. A small error in a large box is generally benign but a small error in a small box has a much greater effect on IOU. Our main source of error is incorrect localizations.

损失函数对待小边框和大边框是平等的。但是一个小的偏差对于大边框来说不算什么，但对于小边框来说会极大地影响IOU。

3. Comparison to Other Detection Systems

Object detection is a core problem in computer vision. Detection pipelines generally start by extracting a set of robust features from input images (Haar [25], SIFT [23], HOG [4], convolutional features [6]). Then, classifiers [36, 21, 13, 10] or localizers [1, 32] are used to identify objects in the feature space. These classifiers or localizers are run either in sliding window fashion over the whole im- age or on some subset of regions in the image [35, 15, 39]. We compare the YOLO detection system to several top de- tection frameworks, highlighting key similarities and differ- ences.

Deformable parts models. Deformable parts models (DPM) use a sliding window approach to object detection [10]. DPM uses a disjoint pipeline to extract static features, classify regions, predict bounding boxes for high scoring regions, etc. Our system replaces all of these disparate parts with a single convolutional neural network. The network performs feature extraction, bounding box prediction, non- maximal suppression, and contextual reasoning all concur- rently. Instead of static features, the network trains the fea- tures in-line and optimizes them for the detection task. Our unified architecture leads to a faster, more accurate model than DPM.

R-CNN. R-CNN and its variants use region proposals in- stead of sliding windows to find objects in images. Selective Search [35] generates potential bounding boxes, a convolu- tional network extracts features, an SVM scores the boxes, a linear model adjusts the bounding boxes, and non-max sup- pression eliminates duplicate detections. Each stage of this complex pipeline must be precisely tuned independently and the resulting system is very slow, taking more than 40 seconds per image at test time [14].

YOLO shares some similarities with R-CNN. Each grid cell proposes potential bounding boxes and scores those boxes using convolutional features. However, our system puts spatial constraints on the grid cell proposals which helps mitigate multiple detections of the same object. Our system also proposes far fewer bounding boxes, only 98 per image compared to about 2000 from Selective Search. Finally, our system combines these individual components into a single, jointly optimized model.

Other Fast Detectors Fast and Faster R-CNN focus on speeding up the R-CNN framework by sharing computa- tion and using neural networks to propose regions instead of Selective Search [14] [28]. While they offer speed and accuracy improvements over R-CNN, both still fall short of real-time performance.

Many research efforts focus on speeding up the DPM pipeline [31] [38] [5]. They speed up HOG computation, use cascades, and push computation to GPUs. However, only 30Hz DPM [31] actually runs in real-time.

Instead of trying to optimize individual components of a large detection pipeline, YOLO throws out the pipeline entirely and is fast by design.

Detectors for single classes like faces or people can be highly optimized since they have to deal with much less variation [37]. YOLO is a general purpose detector that learns to detect a variety of objects simultaneously.
Deep MultiBox. Unlike R-CNN, Szegedy et al. train a convolutional neural network to predict regions of interest [8] instead of using Selective Search. MultiBox can also perform single object detection by replacing the confidence prediction with a single class prediction. However, Multi- Box cannot perform general object detection and is still just a piece in a larger detection pipeline, requiring further im- age patch classification. Both YOLO and MultiBox use a convolutional network to predict bounding boxes in an im- age but YOLO is a complete detection system.

OverFeat. Sermanet et al. train a convolutional neural network to perform localization and adapt that localizer to perform detection [32]. OverFeat efficiently performs slid- ing window detection but it is still a disjoint system. Over- Feat optimizes for localization, not detection performance. Like DPM, the localizer only sees local information when making a prediction. OverFeat cannot reason about global context and thus requires significant post-processing to pro- duce coherent detections.

MultiGrasp. Our work is similar in design to work on grasp detection by Redmon et al [27]. Our grid approach to bounding box prediction is based on the MultiGrasp system for regression to grasps. However, grasp detection is a much simpler task than object detection. MultiGrasp only needs to predict a single graspable region for an image containing one object. It doesn’t have to estimate the size, location, or boundaries of the object or predict it’s class, only find a region suitable for grasping. YOLO predicts both bounding boxes and class probabilities for multiple objects of multi- ple classes in an image.